Take April and as City New York City

In [52]:
import pandas as pd
import numpy as np
import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import lars_path
from sklearn.linear_model import LinearRegression, Lasso, LassoCV
from sklearn.metrics import r2_score
import scipy.stats as stats
import matplotlib.pyplot as plt
import math
%matplotlib inline

In [53]:
airbnbOCT= pd.read_csv("/Users/jacopobinati/Desktop/HM2/listings_apr.csv")
airbnbOCT.head()

/var/folders/6t/d3d0rh5d1rd426bk9g_mnhd00000gn/T/ipykernel_14578/3853903799.py:1: DtypeWarning: Columns (68) have mixed types. Specify dtype option on import or set low_memory=False.
  airbnbOCT= pd.read_csv("/Users/jacopobinati/Desktop/HM2/listings_apr.csv")


,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,765948794133787266,https://www.airbnb.com/rooms/765948794133787266,20230405045506,2023-04-05,city scrape,Brooklyn Refuge,Take a break and unwind at this peaceful oasis.,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,488760226,...,4.89,4.72,4.94,NaN,t,2,1,1,0,4.46
1,13060365,https://www.airbnb.com/rooms/13060365,20230405045506,2023-04-06,previous scrape,Private Bedroom in 1800 Sq Ft loft,Quiet furnished private bedroom in a 5 BDR Tri...,Tribeca is one of the best neighborhoods in Ma...,https://a0.muscache.com/pictures/49b803fe-9c5a...,5252741,...,NaN,NaN,NaN,NaN,f,1,0,1,0,0.01
2,13316028,https://www.airbnb.com/rooms/13316028,20230405045506,2023-04-06,previous scrape,Duplex Condo in Williamsburg- Pvt Ensuite Bath!,"A bright, airy hideaway located on the norther...",NaN,https://a0.muscache.com/pictures/71edd41f-54f6...,6459374,...,5.00,4.75,5.00,NaN,f,1,0,1,0,0.11
3,13308276,https://www.airbnb.com/rooms/13308276,20230405045506,2023-04-06,previous scrape,The Manhattan - Apartment Near Central Park,Only 1 block from Central Park and 2 blocks fr...,The Upper West Side is filled with beautifully...,https://a0.muscache.com/pictures/7792cacc-e141...,75085688,...,4.70,4.60,4.90,NaN,f,1,1,0,0,0.12
4,13355711,https://www.airbnb.com/rooms/13355711,20230405045506,2023-04-06,previous scrape,Private room in Washington Heights,You’ll love my place because of the neighborho...,NaN,https://a0.muscache.com/pictures/e2822c2f-a54b...,39644709,...,5.00,4.00,5.00,NaN,f,1,0,1,0,0.01


In [54]:
column_info = pd.DataFrame({
    "ID": range(len(airbnbOCT.columns)),
    "Variable": airbnbOCT.columns
})
print(column_info)

    ID                                      Variable
0    0                                            id
1    1                                   listing_url
2    2                                     scrape_id
3    3                                  last_scraped
4    4                                        source
..  ..                                           ...
70  70                calculated_host_listings_count
71  71   calculated_host_listings_count_entire_homes
72  72  calculated_host_listings_count_private_rooms
73  73   calculated_host_listings_count_shared_rooms
74  74                             reviews_per_month

[75 rows x 2 columns]


### Cleaning Dataset
#### After indexing the columns, i dropped some of them (can be checked under). Moreover, I decided to keep a threshold of 40% for varibales with NAs. If we are too narrow, we will reduce the number of variable drastically.

In [55]:
columns_to_drop = [
    0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
    11, 12, 13, 15, 20,
    21,
    44, 45, 46, 47,
    56, 60,
    61, 69
]
airbnb_dropped = airbnbOCT.drop(columns=airbnbOCT.columns[columns_to_drop])
threshold = 0.4
airbnb_nmiss = airbnb_dropped.loc[:, airbnb_dropped.isnull().mean() < threshold]
print(airbnb_nmiss.describe())
airbnb2 = airbnb_nmiss.copy()
airbnb2['price'] = airbnb2['price'].replace('[\$,]', '', regex=True).astype(float)

airbnb2['price'] = pd.to_numeric(airbnb2['price'], errors='coerce')
airbnb2 = airbnb2[airbnb2['price'] <= 10000]
airbnb2['ln_price'] = np.log(airbnb2['price'])

       host_listings_count  host_total_listings_count      latitude   
count         43236.000000               43236.000000  43241.000000  \
mean            114.687136                 173.536613     40.728693   
std             592.050443                 832.501414      0.057481   
min               1.000000                   1.000000     40.500314   
25%               1.000000                   1.000000     40.687610   
50%               2.000000                   3.000000     40.724430   
75%               5.000000                   8.000000     40.762510   
max            4815.000000                8456.000000     40.911380   

          longitude  accommodates      bedrooms          beds  minimum_nights   
count  43241.000000  43241.000000  39365.000000  42356.000000    43241.000000  \
mean     -73.943633      2.947018      1.411965      1.646992       18.205314   
std        0.056330      2.109676      0.810242      1.144478       27.104094   
min      -74.249840      0.000000   

/Users/jacopobinati/opt/miniconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [56]:
for column in airbnb2.columns:
    value_type = type(airbnb2.loc[0, column])
    print(f"Column '{column}' has values of type: {value_type}")

Column 'host_response_rate' has values of type: <class 'str'>
Column 'host_acceptance_rate' has values of type: <class 'str'>
Column 'host_is_superhost' has values of type: <class 'str'>
Column 'host_thumbnail_url' has values of type: <class 'str'>
Column 'host_listings_count' has values of type: <class 'numpy.float64'>
Column 'host_total_listings_count' has values of type: <class 'numpy.float64'>
Column 'host_verifications' has values of type: <class 'str'>
Column 'host_has_profile_pic' has values of type: <class 'str'>
Column 'host_identity_verified' has values of type: <class 'str'>
Column 'neighbourhood_cleansed' has values of type: <class 'str'>
Column 'neighbourhood_group_cleansed' has values of type: <class 'str'>
Column 'latitude' has values of type: <class 'numpy.float64'>
Column 'longitude' has values of type: <class 'numpy.float64'>
Column 'property_type' has values of type: <class 'str'>
Column 'room_type' has values of type: <class 'str'>
Column 'accommodates' has values o

In [58]:
airbnb2.drop(columns=['room_type'], inplace=True)
print(airbnb2['review_scores_value'].value_counts)

<bound method IndexOpsMixin.value_counts of 0        4.94
1         NaN
2        5.00
3        4.90
4        5.00
         ... 
43236    5.00
43237    4.91
43238    5.00
43239    4.90
43240    4.67
Name: review_scores_value, Length: 43235, dtype: float64>


Regarding the property type: we merged observations which falls into the category of Private Room, House and Shared Room. Then all the other observations have been dropped.

In [59]:
airbnb2['property_type'] = airbnb2['property_type'].apply(lambda x: 'Private room' if x.startswith('Private room') else x)
airbnb2['property_type'] = airbnb2['property_type'].apply(lambda x: 'Private room' if x.startswith('Room') else x)
airbnb2['property_type'] = airbnb2['property_type'].apply(lambda x: 'House' if x.startswith('Entire') else x)
airbnb2['property_type'] = airbnb2['property_type'].apply(lambda x: 'Shared Room' if x.startswith('Shared') else x)
property_type_counts = airbnb2['property_type'].value_counts().reset_index()
property_type_counts.columns = ['Property_Type', 'Count']
column_info2 = pd.DataFrame({
    "ID": range(len(airbnbOCT.property_type)),
    "Variable": airbnbOCT.property_type
})
column_info2

allowed_types = ['Private room', 'House', 'Shared Room']
airbnb2 = airbnb2[airbnb2['property_type'].isin(allowed_types)]
property_type_counts

,Property_Type,Count
0,House,24164
1,Private room,18410
2,Shared Room,586
3,Casa particular,19
4,Tiny home,13
5,Camper/RV,12
6,Floor,8
7,Houseboat,7
8,Boat,4
9,Tower,3


In [60]:

property_dummies = pd.get_dummies(airbnb2['property_type'], prefix='property', drop_first=False)
airbnb2 = pd.concat([airbnb2, property_dummies], axis=1)
airbnb2

,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,...,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,ln_price,property_House,property_Private room,property_Shared Room
0,100%,100%,f,https://a0.muscache.com/im/pictures/user/87e00...,2.0,5.0,"['email', 'phone']",t,t,Canarsie,...,4.94,2,1,1,0,4.46,3.401197,False,True,False
1,NaN,NaN,f,https://a0.muscache.com/im/pictures/user/dbdbd...,1.0,5.0,"['email', 'phone']",t,t,Civic Center,...,NaN,1,0,1,0,0.01,3.951244,False,True,False
2,NaN,NaN,f,https://a0.muscache.com/im/pictures/user/2b9ba...,1.0,1.0,"['email', 'phone']",t,t,Greenpoint,...,5.00,1,0,1,0,0.11,4.787492,False,True,False
3,NaN,NaN,f,https://a0.muscache.com/im/pictures/user/38b3d...,1.0,1.0,"['email', 'phone']",t,f,Upper West Side,...,4.90,1,1,0,0,0.12,5.010635,True,False,False
4,NaN,NaN,f,https://a0.muscache.com/im/pictures/user/9d5f1...,1.0,1.0,"['email', 'phone']",t,t,Washington Heights,...,5.00,1,0,1,0,0.01,3.218876,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43236,100%,97%,f,https://a0.muscache.com/im/pictures/user/947ca...,1.0,4.0,"['email', 'phone']",t,t,Fresh Meadows,...,5.00,1,1,0,0,2.00,4.317488,True,False,False
43237,80%,68%,f,https://a0.muscache.com/im/pictures/user/595de...,1.0,1.0,"['email', 'phone']",t,t,Flushing,...,4.91,1,1,0,0,0.51,4.499810,True,False,False
43238,100%,60%,f,https://a0.muscache.com/im/pictures/user/74d2d...,2.0,2.0,['phone'],t,t,Parkchester,...,5.00,2,0,2,0,0.18,4.510860,False,True,False
43239,100%,97%,t,https://a0.muscache.com/im/pictures/user/e43fb...,1.0,1.0,['phone'],t,t,Edenwald,...,4.90,1,0,1,0,1.57,3.970292,False,True,False


### Dealing with Amenities

In [8]:
print(airbnb2['amenities'].head(1))

amenities = airbnb2['amenities'].str.split(',')
amenities_table = pd.Series([item for sublist in amenities for item in sublist]).value_counts()
sorted_table = amenities_table.sort_values(ascending=False)
print(sorted_table.head())

0    ["TV", "Wifi", "Kitchen", "Carbon monoxide ala...
Name: amenities, dtype: object
 "Wifi"                     38023
 "Kitchen"                  37869
 "Essentials"               35073
 "Carbon monoxide alarm"    33122
 "Hangers"                  29656
Name: count, dtype: int64


### Transform amenities into a binary indicator matrix

In [9]:
amenities_table = airbnb2['amenities'].str.get_dummies(sep=',')
print(amenities_table.head())
amenities_table.describe()

     Bodywash body soap"    GOOD SOAP brand    R&Co    body soap"    shampoo"   
0                      0                  0       0             0           0  \
1                      0                  0       0             0           0   
2                      0                  0       0             0           0   
3                      0                  0       0             0           0   
4                      0                  0       0             0           0   

    " Bertazzoni stainless steel oven"   
0                                    0  \
1                                    0   
2                                    0   
3                                    0   
4                                    0   

    " Bose Speaker Bluetooth sound system"   
0                                        0  \
1                                        0   
2                                        0   
3                                        0   
4                             

,"Bodywash body soap""",GOOD SOAP brand,R&Co,"body soap""","shampoo""",""" Bertazzoni stainless steel oven""",""" Bose Speaker Bluetooth sound system""",""" Bowers & Wilkins sound system""",""" Eco Botanics Travel Size conditioner""",""" Eco Botanics Travel Size shampoo""",...,"[""TV""","[""Target brand conditioner""","[""Terra Pure body soap""","[""Washer""","[""Wifi""","[""Wifi""]","[""Window guards""",[],"dove body soap""","summers eve for Women body soap""]"
count,43235.000000,43235.000000,43235.000000,43235.000000,43235.000000,43235.000000,43235.000000,43235.000000,43235.000000,43235.000000,...,43235.000000,43235.000000,43235.000000,43235.000000,43235.000000,43235.000000,43235.000000,43235.000000,43235.000000,43235.000000
mean,0.000023,0.000023,0.000023,0.000046,0.000023,0.000023,0.000023,0.000023,0.000023,0.000023,...,0.057453,0.000023,0.000023,0.000763,0.036267,0.000185,0.000069,0.003215,0.000023,0.000023
std,0.004809,0.004809,0.004809,0.006801,0.004809,0.004809,0.004809,0.004809,0.004809,0.004809,...,0.232710,0.004809,0.004809,0.027617,0.186956,0.013602,0.008330,0.056610,0.004809,0.004809
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Top 10 Amenities in New York City

In [10]:
amenities_nyc = airbnb2['amenities'].str.split(',')
amenities_table_nyc = pd.Series([item for sublist in amenities_nyc for item in sublist]).value_counts()

# Top 10 amenities in NYC
top_10_nyc_amenities = amenities_table_nyc.head(10)
print(top_10_nyc_amenities)

 "Wifi"                     38023
 "Kitchen"                  37869
 "Essentials"               35073
 "Carbon monoxide alarm"    33122
 "Hangers"                  29656
 "Air conditioning"         28223
 "Hot water"                27452
 "Smoke alarm"]             27288
 "Hair dryer"               27158
 "Iron"                     26791
Name: count, dtype: int64


In [11]:
amenities = [col for col in airbnb2 if col.startswith("d_")]

In [12]:
variable_names = airbnb2.columns.tolist()
variable_table = pd.DataFrame(variable_names, columns=["Variable Names"])
pd.set_option('display.max_rows', None)
print(variable_table)

                                  Variable Names
0                             host_response_rate
1                           host_acceptance_rate
2                              host_is_superhost
3                             host_thumbnail_url
4                            host_listings_count
5                      host_total_listings_count
6                             host_verifications
7                           host_has_profile_pic
8                         host_identity_verified
9                         neighbourhood_cleansed
10                  neighbourhood_group_cleansed
11                                      latitude
12                                     longitude
13                                 property_type
14                                     room_type
15                                  accommodates
16                                bathrooms_text
17                                      bedrooms
18                                          beds
19                  

In [13]:
airbnb2.to_csv('/Users/jacopobinati/Desktop/HM2/listings_APR_FINAL.csv', index=False)